# One-way Repeated Measures ANOVA
In the previous part of this lesson, we examined the paired $t$-test as the most basic method for dealing with repeated measurements. In doing do, we established that taking the *difference* between the pairs was enough to remove correlation and render the data independent, allowing us to specify a simple one-sample test on the mean difference. In order to understand this, we split our usual single error term into two components, one associated with each subject and one associated with any other deviations. We then saw how the term associated with each subject *cancelled* when the pairs were subtracted, implying that this component captured the correlation between the repeated measurements. Furthermore, by viewing these terms in relation to a partition of the error variance, we established that it is the *between-subjects* variance that is removed by subtraction, leaving only the *within-subject* variance. Because this is smaller than the total variance, and smaller than the between-subjects variance, this rendered the standard error of the estimated mean difference smaller, thus making the test statistic larger. This not only established *why* the paired $t$-test is different to the independent measures $t$-test, but also why the paired $t$-test is typically *more powerful* and thus desirable, from an inferential perspective. 

## Limitations of the Model of *Paired Differences*
Although this *paired differences* framework works well when the data consists of only *two* repeated measurements, we hit a problem if we have *more than two* repeated measurements, *more than one* repeated measures factor or even when we have a mixture of repeated measurements and independent measurements. In many of these cases, we cannot take a simple subtraction and then analyse the resultant differences. So, while the paired $t$-test is fine in simple cases, it does not generalise very easily. 

In order to allow the analysis of more complex experimental situations, we need to abandon the idea of making the data independent through subtraction. Instead, we need to accommodate the correlation *directly* within the model. The general theme of this section of the unit is *mixed-effects models*, which represent the most modern solution to this problem. However, in this part of the lesson we will discuss an older solution in the form of the *repeated measures ANOVA*. This is really a stepping-stone towards mixed-effects models, rather than a recommendation, as the repeated measures ANOVA is rather limited in practice. Nevertheless, this method is still used widely in psychological research and so is still worth understanding.

## The Model of *Partitioned Errors*
As mentioned above, because *subtraction* only works in a number of limited cases, we need to abandon it as our general solution to dealing with dependent data. Instead, we need to work with data that we *know* is correlated. Unfortunately, as we have already established, the linear model framework assumes that the errors of the model are $i.i.d.$, which will not be true under repeated measurements. However, we have already seen a possible solution to this problem, as the removal of the subject terms $S_{i}$ theoretically *removes* the correlation and renders the errors independent. Thus, if we were to *include* the term $S_{i}$ in the model, the errors would meet the $i.i.d.$ criteria. Furthermore, the single error variance assumed by the linear model would be the *within-subject* variance and thus would be suitable for inference on the repeated measurements. Putting all this together gives us the model

$$
\begin{alignat*}{1}
    y_{ij}    &=                      \mu + \alpha_{j} + S_{i} + \eta_{ij} \\
    \eta_{ij} &\overset{i.i.d.}{\sim} \mathcal{N}(0,\sigma^{2}_{w})
\end{alignat*}
$$

which is the basis for the repeated measures ANOVA.

Now, in the specification above, we have treated $S_{i}$ like any other factor in an ANOVA model. However, this is not really correct. As discussed in the previous part of this lesson, $S_{i}$ comes from *splitting* the overall error term $\epsilon_{ij}$. So, in theory, rather than representing population constants (like $\mu$ and $\alpha_{j}$), the $S_{i}$ are a *random variable*. This makes sense because the subjects represent a *random sample*, rather than a fixed quantity. If we were to run the experiment again, $\mu$ and $\alpha_{j}$ would be the *same*, but the $S_{i}$ would be *different*. Rather than $S_{i}$ representing the $i$th level of an $n$-dimensional experimental factor, it is the $i$th *random deviation*. From this perspective, $S_{i}$ is clearly an *additional error term*. Unfortunately, the linear model only has *one* error term. Thus, what we *want* to use is the model

$$
\begin{alignat*}{1}
    y_{ij}    &=    \mu + \alpha_{j} + S_{i} + \eta_{ij} \\
    S_{i}     &\sim \mathcal{N}(0,\sigma^{2}_{b})        \\
    \eta_{ij} &\sim \mathcal{N}(0,\sigma^{2}_{w})
\end{alignat*}
$$

but we cannot, because this would require a method that could flexibly accommodate multiple error terms (which is exactly what *mixed-effects* model do). Instead, the repeated measures ANOVA aims to *replicate* this situation within the confines of a modelling framework that *does not allow it*. As you might imagine, this involves jumping through a number of hoops and places a number of restrictions on what is possible.

### Partitioned Errors in `R`
Before discussing this in more detail, we will demonstrate the general idea in `R` in two ways, one using `lm()` and one using the `aov()` function. To begin with, let us specify the same model given above that contains the $S_{i}$ term. We will do this using the `mice2` data again, so we can show agreement with the paired $t$-test. 

First, we need to first convert this data into *long* format.

In [1]:
library('datarium')
library('reshape2')
data('mice2')

# repeats and number of subjects
t <- 2
n <- dim(mice2)[1]

# reshape wide -> long
mice2.long <- melt(mice2,                       # wide data frame
                   id.vars='id',                # what stays fixed?
                   variable.name="time",        # name for the new predictor
                   value.name="weight")         # name for the new outcome

mice2.long <- mice2.long[order(mice2.long$id),] # order by ID
rownames(mice2.long) <- seq(1,n*t)              # fix row names

print(mice2.long)

   id   time weight
1   1 before  187.2
2   1  after  429.5
3   2 before  194.2
4   2  after  404.4
5   3 before  231.7
6   3  after  405.6
7   4 before  200.5
8   4  after  397.2
9   5 before  201.7
10  5  after  377.9
11  6 before  235.0
12  6  after  445.8
13  7 before  208.7
14  7  after  408.4
15  8 before  172.4
16  8  after  337.0
17  9 before  184.6
18  9  after  414.3
19 10 before  189.6
20 10  after  380.3


Next, we need to turn `id` into a factor

In [2]:
mice2.long$id <- as.factor(mice2.long$id)

Finally, we can specify the partitioned error model using `lm()` below

In [3]:
rm.mod <- lm(weight ~ time + id, data=mice2.long)
summary(rm.mod)


Call:
lm(formula = weight ~ time + id, data = mice2.long)

Residuals:
    Min      1Q  Median      3Q     Max 
-21.410  -7.155   0.000   7.155  21.410 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  208.610     12.949  16.110 6.06e-08 ***
timeafter    199.480      7.809  25.546 1.04e-09 ***
id2           -9.050     17.460  -0.518   0.6167    
id3           10.300     17.460   0.590   0.5698    
id4           -9.500     17.460  -0.544   0.5996    
id5          -18.550     17.460  -1.062   0.3157    
id6           32.050     17.460   1.836   0.0996 .  
id7            0.200     17.460   0.011   0.9911    
id8          -53.650     17.460  -3.073   0.0133 *  
id9           -8.900     17.460  -0.510   0.6225    
id10         -23.400     17.460  -1.340   0.2130    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 17.46 on 9 degrees of freedom
Multiple R-squared:  0.987,	Adjusted R-squared:  0.9725 
F-statistic: 68.

Ignoring the `id` effects, if we look at the test on `timeafter`, we can see that this agrees with the paired $t$-test from earlier. We can also tidy this output up a bit by calling `Anova()` on the model

In [4]:
library('car')
print(Anova(rm.mod))

Loading required package: carData
Anova Table (Type II tests)

Response: weight
          Sum Sq Df F value    Pr(>F)    
time      198961  1 652.613 1.039e-09 ***
id          9013  9   3.285   0.04559 *  
Residuals   2744  9                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Where, again, we would ignore the test on `id` and just focus on the effect of `time`. 

Now, we have managed to produce correct tests for our factor of interest, so our model specification must be correct in that respect. However, the *way* we have done this needs some thought. After all, the fact that tests on `id` has been produced suggests that this model term is not being treated correctly. Do we really want a hypothesis test based on the null that all the subject means are the same? Given that this is a random sample of subjects, how meaningful would this be unless these were the only subjects in the world that we were interested in? As indicated earlier, the `id` effect is not really an effect of interest in the same way as `time`. In fact, `id` is considered part of the *error* rather than part of the *mean*. The way we have included `id` above is as part of the *mean function*, but really `id` should be part of the *variance function*. It would be like trying to specify a hypothesis test on the model residuals. We are not interested in some universal truth here because what we have are *random deviations* reflective of *error variance*. As such, in much the same way that the residuals are used to estimate the *within-subject* variance, we want the values of `id` to be used to estimate the *between-subjects* variance. Clearly, all we have done above is *remove* the variance of `id` from the residuals by moving it into the model equation. But we have done nothing to suggest that `id` should be treated differently from `time`. 

In order to tell `R` that `id` is an *additional error term*, we can use the `aov()` function. This is a wrapper for `lm()` designed to produce an ANOVA table where some of the model terms represent *error* rather than a traditional ANOVA effect. From an *estimation* perspective, this distinction does not make any difference because ANOVA mean squares are *already* estimates of variance. However, what *does* matter, is whether these mean squares form the *numerator* of an $F$-ratio (variance associated with *mean differences*) or the *denominator* of an $F$-ratio (variance associated with *error*). So, what we are really doing is telling `R` where to place the different mean squares in the ANOVA table. In other words, we have to tell `R` how we want the arithmetic to be organised.

An example of using `aov()` for the `mice2` data is shown below. Here we use the `Error()` syntax within the model formula to indicate that `id` is an *error term*. This results in *two* ANOVA tables. One where $S_{i}$ forms the error term and one where $\eta_{ij}$ forms the error term. Based on the model structure, `aov()` can work out that `time` should be in the table with $\sigma^{2}_{w}$ as the error term. As there are no other terms in the model, the ANOVA table with $id$ as the error is empty, but we will see examples where this is not the case further below.

In [5]:
rm.aov.mod <- aov(weight ~ time + Error(id), data=mice2.long)
print(summary(rm.aov.mod))


Error: id
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals  9   9013    1002               

Error: Within
          Df Sum Sq Mean Sq F value   Pr(>F)    
time       1 198961  198961   652.6 1.04e-09 ***
Residuals  9   2744     305                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


The main conclusion to draw at present is that the repeated measures ANOVA is based on a *partitioned error* model, where we separate the variance into $\sigma^{2} = \sigma^{2}_{b} + \sigma^{2}_{w}$. However, because the normal linear model only has a *single* error term, we cannot do this automatically. Instead, we have to *remove* additional sources of variance from the errors by placing them within the *mean function*. We then have to manually construct different ANOVA tables for the different sources, using the mean square of some terms as *denominators* rather than *numerators*. This means that our *estimated model* is based only on a single error term, but the ANOVA table is based on treating some of the terms in the mean function as *additional error terms*. Hopefully it is clear that this is effectively an *ad hoc bolt-on* to a procedure that was never designed to support it[^submodel-foot].

## The Implied Covariance Structure
Another important element of the repeated measures ANOVA is the implied covariance structure. Although a variance-covariance matrix is never actually estimated or constructed as part of this procedure, it is *implied* by the model assumptions. As we will see further below, this implied structured is actually very restrictive and makes some strong claims about the nature of the repeated measurements. To see this, first consider the partitioned error model that the ANOVA table is assuming


$$
\begin{alignat*}{2}
    y_{ij}             &\sim \mathcal{N}(\mu_{j}, \sigma^{2})             &\quad \text{(Population distribution)} \\
    E(y_{ij})          &=    \mu_{j} = \mu + \alpha_{j}                   &\quad \text{(Mean function)}           \\
    \text{Var}(y_{ij}) &=    \sigma^{2} = \sigma^{2}_{b} + \sigma^{2}_{w} &\quad \text{(Variance function)}.      \\
\end{alignat*}
$$

So, we can see that each observation is assumed to have the same variance, formed from $\sigma^{2}_{b} + \sigma^{2}_{w}$. This gives us the implied *diagonal* elements of the variance-covariance matrix. What about the *off-diagonal* elements? In a normal linear model these would be 0 to indicate no correlation. However, because we have repeated measurements, this will not be true. As established earlier, the fact that some observations share the error term $S_{i}$ creates correlation. The way this works is by examining the implied *covariance* between two measurements from the same subject

$$
\text{Cov}(y_{i1},y_{i2}) = \text{Cov}(\mu + \alpha_{1} + S_{i} + \epsilon_{i1}, \mu + \alpha_{2} + S_{i} + \epsilon_{i2}) 
$$

Because $\mu$, $\alpha_{1}$ and $\alpha_{2}$ are *population constants*, they have 0 variance and thus do not contribute to the definition of covariance. These terms therefore drop-out, leaving

$$
\text{Cov}(y_{i1},y_{i2}) = \text{Cov}(S_{i} + \epsilon_{i1}, S_{i} + \epsilon_{i2}) 
$$

This can be expanded like so

$$
\text{Cov}(y_{i1},y_{i2}) = \text{Cov}(S_{i},S_{i}) + \text{Cov}(S_{i},\epsilon_{i2}) + \text{Cov}(\epsilon_{i1},S_{i}) + \text{Cov}(\epsilon_{i1}, \epsilon_{i2}). 
$$

The subject effects and the errors are not correlated as these represent independent partitions of the overall error. As such, $\text{Cov}(S_{i},\epsilon_{i2}) = \text{Cov}(\epsilon_{i1},S_{i}) = 0$. Similarly, the final errors are uncorrelated because the correlation has been *removed* by partitioning-out the subject effects. So $\text{Cov}(\epsilon_{i1}, \epsilon_{i2}) = 0$. This leaves

$$
\text{Cov}(y_{i1},y_{i2}) = \text{Cov}(S_{i},S_{i}).
$$

A key result from the definition of covariance is that the [covariance of a random variable with itself is simply its variance](https://en.wikipedia.org/wiki/Covariance#Covariance_with_itself), meaning

$$
\text{Cov}(y_{i1},y_{i2}) = \text{Cov}(S_{i},S_{i}) = \text{Var}(S_{i}) = \sigma^{2}_{b}.
$$

All of which is to say that the variance associated with the subject-specific deflections *is* the covariance induced by the repeated measurements. This means that the repeated measures ANOVA is implying the following covariance structure (as shown for two repeated measurements from two subjects)

$$
\begin{bmatrix}
    y_{11} \\
    y_{12} \\
    y_{21} \\
    y_{22} \\
\end{bmatrix}
\sim\mathcal{N}\left(
\begin{bmatrix}
    \mu + \alpha_{1} \\
    \mu + \alpha_{2} \\
    \mu + \alpha_{1} \\
    \mu + \alpha_{2} \\
\end{bmatrix}, 
\begin{bmatrix}
    \sigma^{2}_{b} + \sigma^{2}_{w} & \sigma^{2}_{b}                  & 0                               & 0                               \\
    \sigma^{2}_{b}                  & \sigma^{2}_{b} + \sigma^{2}_{w} & 0                               & 0                               \\
    0                               & 0                               & \sigma^{2}_{b} + \sigma^{2}_{w} & \sigma^{2}_{b}                  \\
    0                               & 0                               & \sigma^{2}_{b}                  & \sigma^{2}_{b} + \sigma^{2}_{w} \\
\end{bmatrix}
\right)
$$

This type of structure is known as *compound symmetry*, because all variances and covariances are the *same* across the repeated measurements. This is not really a problem when we have two repeats, because there is only one covariance term. However, consider what happens if we were to have *three* repeated measurements per-subject

$$
\begin{bmatrix}
    y_{11} \\
    y_{12} \\
    y_{13} \\
    y_{21} \\
    y_{22} \\
    y_{23} \\
\end{bmatrix}
\sim\mathcal{N}\left(
\begin{bmatrix}
    \mu + \alpha_{1} \\
    \mu + \alpha_{2} \\
    \mu + \alpha_{3} \\
    \mu + \alpha_{1} \\
    \mu + \alpha_{2} \\
    \mu + \alpha_{3} \\
\end{bmatrix}, 
\begin{bmatrix}
    \sigma^{2}_{b} + \sigma^{2}_{w}  & \sigma^{2}_{b}                  & \sigma^{2}_{b}                  & 0                                & 0                               & 0                               \\
    \sigma^{2}_{b}                   & \sigma^{2}_{b} + \sigma^{2}_{w} & \sigma^{2}_{b}                  & 0                                & 0                               & 0                               \\
    \sigma^{2}_{b}                   & \sigma^{2}_{b}                  & \sigma^{2}_{b} + \sigma^{2}_{w} & 0                                & 0                               & 0                               \\
    0                                & 0                               & 0                               & \sigma^{2}_{b} + \sigma^{2}_{w}  & \sigma^{2}_{b}                  & \sigma^{2}_{b}                  \\
    0                                & 0                               & 0                               & \sigma^{2}_{b}                   & \sigma^{2}_{b} + \sigma^{2}_{w} & \sigma^{2}_{b}                  \\
    0                                & 0                               & 0                               & \sigma^{2}_{b}                   & \sigma^{2}_{b}                  & \sigma^{2}_{b} + \sigma^{2}_{w} \\
\end{bmatrix}
\right)
$$

This looks messy, but notice that the correlation between each of the repeated measurements is *exactly* the same value. Irrespective of what the experimental conditions are, the repeated measures ANOVA implies that they are correlated in exactly the same way. This is a *big* assumption. What if two conditions are positively correlated, but the others are negatively correlated? What is one of the experimental conditions works by making the correlation *weaker* compared to the other conditions? Any of these more complex situations are effectively *ignore* in the repeated measures ANOVA. Indeed, look back at the correlation structure we saw earlier in the lesson for the `selfesteem` data. This is clearly not compound symmetric. Remember that the importance of covariance relates to when we want to look at the *difference* between any of these conditions. If the covariance structure is *wrong* then the assumed standard error will be *wrong*. If we are using NHST, this means the calculated test statistic will not follow the assumed null distribution and $p$-value will either be *too liberal* or *too conservative*.

### The Sphericity Condition
Although we have suggested that the repeated measures ANOVA assumes a *compound symmetric* covariance structure, the reality is bit more nuanced. As indicated above, the assumption of compound symmetry is tied directly to the calculation of the standard errors, which then impact the denominator of the test statistic and inform the shape of the null distribution used to derive the $p$-values. If the actual covariance structure is *not* compound symmetric, then the formula used for the standard errors will be wrong, as will the formula used for the denominator of the test statistic. This will mean that the null distribution of the statistic is not what we think it is and the $p$-values will be inaccurate. This is a core limitation of the repeated measures ANOVA which is tied directly to the fact that the covariance structure is never explicitly *estimated*, it is only *assumed*.

However, this assumption can actually be *weakened* slightly. Rather than assuming compound symmetry, a related condition known as *sphericity* is all that is required for the calculated $F$-statistic to follow the assumed null distribution. In brief, this condition is that the variance of all *pairwise differences* between the repeated measurements have equal variance. For instance, in the case of 3 repeated measures, this would be

$$
\text{Var}(y_{i1} - y _{i2}) = \text{Var}(y_{i1} - y _{i3}) = \text{Var}(y_{i2} - y _{i3}) = \sigma^{2}.
$$

We can *visualise* why this is known as *sphericity*. In the plots below, we transform the 3 repeated measures from each subject into the 3 subtractions from the formula above. We then plot these against each other in a 3D scatterplot. When sphericity is met (the *left* plot), this creates a *sphere* of points, because the variance is equal in all directions. When sphericity is *not* met (the *right* plot), this creates an *ellipsis*, because the variance is larger for some of these differences compared to others. 

In [6]:
library(MASS)
library(rgl)
library(htmlwidgets)

set.seed(123)
n_subj <- 800

# --- Covariances in contrast space ---
Sigma_spherical <- diag(3)

A <- matrix(c(1,   0,   0,
              0.8, 0.5, 0,
              0.4, 0.3, 0.2),
            nrow = 3, byrow = TRUE)
Sigma_elliptical <- A %*% t(A)

C_sph <- mvrnorm(n = n_subj, mu = c(0, 0, 0), Sigma = Sigma_spherical)
C_ell <- mvrnorm(n = n_subj, mu = c(0, 0, 0), Sigma = Sigma_elliptical)

colnames(C_sph) <- c("C1", "C2", "C3")
colnames(C_ell) <- c("C1", "C2", "C3")

# --- Build the rgl scene in an offscreen device ---
open3d(useNULL = TRUE)
bg3d("white")

layout3d(matrix(1:2, nrow = 1))

# Left: spherical
next3d()
plot3d(
  C_sph[, 1], C_sph[, 2], C_sph[, 3],
  xlab = "yi1 - yi2",
  ylab = "yi1 - yi3",
  zlab = "yi2 - yi3",
  col  = "dodgerblue",
  size = 4,
  alpha = 0.5,
  box = TRUE,
  type = "s"
)
lines3d(c(-4, 4), c(0, 0), c(0, 0), lwd = 3)
lines3d(c(0, 0), c(-4, 4), c(0, 0), lwd = 3)
lines3d(c(0, 0), c(0, 0), c(-4, 4), lwd = 3)

aspect3d("iso") 

# Right: elliptical
next3d()
plot3d(
  C_ell[, 1], C_ell[, 2], C_ell[, 3],
  xlab = "yi1 - yi2",
  ylab = "yi1 - yi3",
  zlab = "yi2 - yi3",
  col  = "firebrick",
  size = 4,
  alpha = 0.5,
  box = TRUE,
  type = "s"
)
lines3d(c(-4, 4), c(0, 0), c(0, 0), lwd = 3)
lines3d(c(0, 0), c(-4, 4), c(0, 0), lwd = 3)
lines3d(c(0, 0), c(0, 0), c(-4, 4), lwd = 3)

aspect3d("iso") 

# Turn scene into widget
w <- rglwidget()

# Save as standalone HTML
saveWidget(w, "_static/sphericity_3d.html", selfcontained = TRUE)


<p style="text-align:center;">
  <iframe
    src="_static/sphericity_3d.html"
    class="sphericity-frame"
    width="800"
    height="600"
    style="border:none;"
  >
  </iframe>
</p>

Now, this condition is really something of a *mathematical trick*. As pointed out by [Davis (2010)](), many authors have suggested that any realistic covariance matrix you may come across in practice that meets the criteria of *sphericity* will also be *compound symmetric*. Indeed, [Wallenstein (1982)]() went as far as to indicate that, for all practical purposes, the assumption of the repeated measures ANOVA *is* compound symmetry. However, the utility of expressing this assumption in terms of sphericity is that it allows for a *correction* to be applied. Rather than defining sphericity as above, it can be expressed in a mathematically equivalent way using a value known as $\epsilon$[^noterr-foot]. When sphericity is met $\epsilon = 1$, and when it is not met $\epsilon \neq 1$. This is useful because $\epsilon$ can be used to adjust the degrees of freedom of the null $F$-distribution to make it correct when sphericity is violated. Thus, a null $F$-distribution of $F(\epsilon \times df_{1}, \epsilon \times df_{2})$ can be used, so long as we know $\epsilon$. Several different methods of estimating $\epsilon$ from the sample covariance matrix have been proposed by authors such as [Greenhouse & Geisser (1959)]() and [Huynh & Feldt (1976)](). 

If you have used statistical software such as SPSS in the past, these corrections should be familiar to you. Within `R`, these are available through the `ezANOVA()` function which, as we will see, is generally the most straight-forward way of generating a repeated measures ANOVA with the correct error partition. For simple models, this will make limited difference. However, as we will see below, more complex models because tricky to specify using `aov()`. This is partly a limitation in the structure of `aov()`, but also points to the general complexity of the repeated measures ANOVA framework.

In the code below, we show fitting a repeated measures ANOVA to the `selfesteem` data, first using `aov()` to correctly partition the error and then using `ezANOVA()` to get the same results with the sphericity corrections.

In [7]:
library('datarium')
library('ez')

## Repeated Measures ANOVA with Corrections in `R`
As we can see above, using a partitioned error model with `aov()` is a tricky business and it would be very easy to get this wrong. As an alternative, we can use the `ezANOVA()` function from the `ez` package. As the name implies, this is designed to allow for an RM ANOVA without the usual difficulties associated with the `aov()` or `lm()` functions. Unfortuantely, the aim of this package is largely to make the `R` output the same as SPSS. So it does away with the linear model framework. This means, no residuals, no parameter estimates, no diagnostic plots or anything else we have made use of so far. If you *have* to use an RM ANOVA, this is the simplest way to get it *right*. However, as we will discuss below, we would disuade you from ever considering RM ANOVA as an option in the future. About the only utility of this is showing doubtful researchers that our better options of GLS and mixed-effects models are, in fact, giving them the same answer as an RM ANOVA.

In [16]:
library(ez)
library('datarium')
data('selfesteem')

#=============================================================================#
# Convert to LONG format
#=============================================================================#
t <- 3
n <- dim(selfesteem)[1]

# reshape wide -> long
selfesteem.long <- melt(selfesteem, id.vars='id',variable.name="time", value.name="score")

selfesteem.long           <- selfesteem.long[order(selfesteem.long$id),] # order by ID
rownames(selfesteem.long) <- seq(1,n*t)                                  # fix row names
selfesteem.long$id        <- as.factor(selfesteem.long$id)               # make id a factor

print(head(selfesteem.long))

#=============================================================================#
# One-way RM ANOVA using aov()
#=============================================================================#
oneway.rm.aov <- aov(score ~ time + Error(id), data=selfesteem.long)
summary(oneway.rm.aov)

  id time    score
1  1   t1 4.005027
2  1   t2 5.182286
3  1   t3 7.107831
4  2   t1 2.558124
5  2   t2 6.912915
6  2   t3 6.308434



Error: id
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals  9   4.57  0.5078               

Error: Within
          Df Sum Sq Mean Sq F value   Pr(>F)    
time       2 102.46   51.23   55.47 2.01e-08 ***
Residuals 18  16.62    0.92                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

So, we can see that the error has been correctly partitioned and we have a suitable test of `time`. However, this is only valid so far as the assumption of sphericity is met. To generate this test as corrected for any violations of sphericity, the easiest approach is to use `ezANOVA()`

In [22]:
oneway.rm.ez <- ezANOVA(data=selfesteem.long, dv=score, wid=id, within=time)
print(oneway.rm.ez)

$ANOVA
  Effect DFn DFd        F            p p<.05       ges
2   time   2  18 55.46903 2.013829e-08     * 0.8285954

$`Mauchly's Test for Sphericity`
  Effect         W          p p<.05
2   time 0.5508534 0.09207551      

$`Sphericity Corrections`
  Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF] p[HF]<.05
2   time 0.6900613 2.161433e-06         * 0.7743711 6.032582e-07         *



Where `GGe` means `Greenhouse-Geiser epsilon` and `HFe` means `Huynh-Feldt epsilon`. These are the estimates of $\epsilon$ using the methods mentioned earlier. As these are both $\hat{\epsilon} \neq 1$, sphericity has been violated. The values `p[GG]` and `p[HF]` give the $p$-values for the omnibus test of `time`, after correcting the degrees of freedom using the two different estimates for $\epsilon$. In both cases this remains significant, irrespective of the sphericity violation.

However, we still have all the limitations of `ezANOVA()` mentioned briefly last semester. Because this function does not aim to use the linear models framework, we cannot generate assumptions plots nor any of the other useful output we would normally get from `lm()`. Similarly, we cannot run follow-up tests using `emmeans`.

[^submodel-foot]: An alternative perspective here is that each error term represents a different *sub-model*. So, we can think of specifying *multiple* models, some of which require us to *average-over* certain factors. For instance, if we were to average-over the repeated measurements and then fit a model on the resultant outcome variable, this model would automatically have $\sigma^{2}_{b}$ as its error term. This does make the whole procedure feel a little bit less of a hack, however, it is very impractical to do this, especially when the number of factors and interactions gets larger. You can read more about this approach in [McFarquhar (2019)](https://www.frontiersin.org/journals/neuroscience/articles/10.3389/fnins.2019.00352/full).

[^noterr-foot]: Note that this is *not* the errors from the linear model, even though the Greek letter is the same.